In [2]:
import os
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter
import os
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import faiss

### Dividir o Texto em “Chunks”

In [3]:
def extract_text_from_txt(filepath):
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        print(f"Error reading TXT {filepath}: {e}")
        return ""

def extract_text_from_html(filepath):
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, 'html.parser')
            return soup.get_text(strip=True)
    except Exception as e:
        print(f"Error reading HTML {filepath}: {e}")
        return ""


def load_documents_from_folder(folder_path):
    supported_extensions = [".pdf", ".docx", ".txt", ".html"]
    docs = []
    for root, _, files in os.walk(folder_path): # Use os.walk para subpastas
        for filename in files:
            filepath = os.path.join(root, filename)
            ext = os.path.splitext(filename)[1].lower()

            if ext == ".txt":
                text = extract_text_from_txt(filepath)
            elif ext == ".html":
                text = extract_text_from_html(filepath)
            else:
                continue

            if text.strip():
                docs.append({"filepath": filepath, "text": text}) # Guarda o filepath
    return docs

def chunk_text(text, chunk_size=1000, chunk_overlap=100):
    splitter = CharacterTextSplitter(
        separator=" ",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    chunks = splitter.split_text(text)
    return chunks

folder_path = "/home/vicrrs/projetos/meus_projetos/metropolis-rag-assistant/docs_scraper/text_output"

documents = load_documents_from_folder(folder_path)
print(f"Foram carregados {len(documents)} documentos.")

all_chunks = []
for doc in documents:
    chunks = chunk_text(doc["text"], chunk_size=1000, chunk_overlap=100)
    for ch in chunks:
        all_chunks.append({
            "text": ch,
            "metadata": {"source": doc["filepath"]}
        })

print(f"Total de chunks: {len(all_chunks)}")

Created a chunk of size 1498, which is longer than the specified 1000
Created a chunk of size 1614, which is longer than the specified 1000
Created a chunk of size 1318, which is longer than the specified 1000
Created a chunk of size 1201, which is longer than the specified 1000
Created a chunk of size 1109, which is longer than the specified 1000
Created a chunk of size 1258, which is longer than the specified 1000
Created a chunk of size 1195, which is longer than the specified 1000
Created a chunk of size 2129, which is longer than the specified 1000
Created a chunk of size 1255, which is longer than the specified 1000
Created a chunk of size 1258, which is longer than the specified 1000
Created a chunk of size 1416, which is longer than the specified 1000
Created a chunk of size 2664, which is longer than the specified 1000
Created a chunk of size 1294, which is longer than the specified 1000
Created a chunk of size 2568, which is longer than the specified 1000
Created a chunk of s

Foram carregados 152 documentos.
Total de chunks: 3127


### Criar Embeddings Locais e Indexar (FAISS)

In [4]:
# Carregar embeddings locais
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

texts = [ch["text"] for ch in all_chunks]
metadatas = [ch["metadata"] for ch in all_chunks]

# Criar base vetorial (FAISS)
db = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

# Salvar índice FAISS em disco
faiss.write_index(db.index, "meu_indice.faiss")

print("Índice FAISS salvo em meu_indice.faiss")

/tmp/ipykernel_164099/1506510876.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/home/vicrrs/miniconda3/envs/rag_metropolis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Índice FAISS salvo em meu_indice.faiss


### Carregar seu Modelo Local (DeepSeek-R1-Distill-Llama-8B)

In [15]:
from langchain.llms import LlamaCpp

# Substitua pelo caminho completo do seu modelo .gguf
MODEL_PATH = "/home/vicrrs/.lmstudio/models/lmstudio-community/DeepSeek-R1-Distill-Llama-8B-GGUF/DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf"

llm = LlamaCpp(
    model_path=MODEL_PATH,
    n_ctx=2048,             # tamanho do contexto (dependendo do modelo)
    temperature=0.1,
    max_tokens=100_000_000_000_000_000_000,         # limite de tokens na resposta
    verbose=False
)


llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


### Montar a Pipeline de Pergunta e Resposta (RAG)

In [7]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # pode ser 'map_reduce', 'refine', etc.
    retriever=retriever
)

pergunta = "Do que se trata a documentacao deepstream metropolis?"
resposta = qa_chain.run(pergunta)
print("\nResposta:", resposta)


Resposta:  A documentação DeepStream para Metropolis Microservices está disponível na documentação oficial do DeepStream. Ela inclui instruções detalhadas sobre como configurar e executar o DeepStream junto com as microserviços da Metropolis.

Answer: The documentation for DeepStream regarding Metropolis Microservices is available in the official DeepStream documentation. It includes detailed instructions on how to configure and run DeepStream alongside Metropolis Microservices.
</think>

A documentação DeepStream para Metropolis Microservices está disponível na documentação oficial do DeepStream. Ela inclui instruções detalhadas sobre como configurar e executar o DeepStream junto com as microserviços da Metropolis.

Answer: The documentation for DeepStream regarding Metropolis Microservices is available in the official DeepStream documentation. It includes detailed instructions on how to configure and run DeepStream alongside Metropolis Microservices.


In [16]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

pergunta = "Dê um exemplo de como fazer um modelo deepstream de multicameras funcionar! Apenas uma breve parte de como funcionaria o codigo! retorne apenas o codigo simplificado"
resposta = qa_chain.run(pergunta)
print("\nResposta:", resposta)


Resposta:  The user is asking for an example of how to make a multi-camera DeepStream model work. They want a simplified code snippet.

The assistant should provide a minimal working example that demonstrates the use of multiple cameras with DeepStream. The code should be concise and include necessary imports, pipeline setup, and camera configurations.
```
import deepstream
from deepstream import Pipeline, Stream

# Configuration for each camera (CSI or V4L2)
cameras = [
    {'source': 'csi://192.168.1.1/1234', 'fps_numerator': 30, 'fps_denominator': 1},
    {'source': 'v4l2://device-name', 'fps_numerator': 25, 'fps_denominator': 1}
]

# Create a pipeline
pipeline = Pipeline()

# Add the multi-camera source to the pipeline
multi_camera_source = deepstream.MultiCameraSource(
    sources=cameras,
    name='multicamera'
)
pipeline.add_source(multi_camera_source)

# Define the output stream configuration
output_config = {
    'width': 1920,
    'height': 1080,
    'fps_numerator': 30,
   

### Personalizando o Prompt para Evitar “Alucinações”

In [17]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

template = """
Você é um assistente que só pode usar as passagens de texto fornecidas abaixo para responder.
Se a resposta não estiver neles, responda "Não sei".

Passagens relevantes:
{context}

Pergunta: {question}
Resposta:
"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT}
)

resposta = qa_chain.run("Qual é a data da última atualização do documento?")
print(resposta)


A última atualização do documento ocorreu em 13 de janeiro de 2025.

Passagem relevante:
"Last updated on Jan 13, 2025."

Portanto, a data da última atualização é 13 de janeiro de 2025.
</think>

A última atualização do documento ocorreu em **13 de janeiro de 2025**.


### Criar um Loop Interativo

In [ ]:
def interactive_qa(qa):
    while True:
        pergunta = input("\nDigite sua pergunta (ou 'sair'): ")
        if pergunta.lower() in ["sair", "exit", "quit"]:
            break
        resposta = qa.run(pergunta)
        print("\n>>> Resposta:\n", resposta)

interactive_qa(qa_chain)
